In [22]:
import sys
sys.path.append('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/')
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import yfinance as yf
import time
from datetime import timedelta, datetime
from systems.performance_reporter import StrategyReporter, PerformanceReporter
from systems.utils import MarketDataExtractor

In [23]:
market_data_extractor = MarketDataExtractor()
strategy_reporter = StrategyReporter()
performance_reporter = PerformanceReporter(market_data_extractor)

In [24]:
# All the Inputs for the Backtest that needs to be analyzed
testname = 'cfb2ec3d1af6d95b97dc0563f9913f547b361bd6703080198201f97c7a7bc4a8_1' # Profits not Reinvested
# testname = '3aa6bc1bc58ab86f2a23419d3737ec62991783e80d8d8802f5cc68a1cfb0a33b' # Profits Reinvested & Margin Deployed @ 0.65 of 3.3x
save_folder = "/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/data/yahoo/1d"
test_folder_path = f'/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/vault/backtest_reports/{testname}'
backtest_output = strategy_reporter.load_backtest_output(test_folder_path)
open_orders = backtest_output['open_orders']
closed_orders = backtest_output['closed_orders']
config_dict = backtest_output['config_dict']

In [25]:
#Processing Open order to make them close, and remove history
open_orders_list = strategy_reporter.clean_up_order_list(open_orders, drop_history=True)
#Processing Closed orders to remove history
closed_orders_list = strategy_reporter.clean_up_order_list(closed_orders, drop_history=True)
#Merge both orders
final_list = closed_orders_list + open_orders_list
# Step 2: Sort the filtered orders by exit order `filled_timestamp`
sorted_orders = sorted(final_list, key=lambda x: x[1]['filled_timestamp'])
# Step 3: Fetch the first and last `filled_timestamp` with buffer
start_date = sorted_orders[0][1]['filled_timestamp'] - timedelta(days=365)
end_date = sorted_orders[-1][1]['filled_timestamp'] + timedelta(days=365)
# end_date = pd.Timestamp(datetime(2010, 1, 1))
# Current we are downloading csv file for below three index
indices = ["^DJI", "^IXIC", "^GSPC"]

index_data_dict = strategy_reporter.load_and_save_index_data(indices, start_date, end_date, save_folder)
rolling_window = int(max(60, (end_date - start_date).days / 20))
print({f"Analyzing with a rolling window of {rolling_window} days"})

{'Analyzing with a rolling window of 312 days'}


### This below function calculates profit/loss from the orders  **(exitPrice - entryPrice) * orderQuantity**

This function also plot the progress lines for 3 indexes provided earlier.

In [26]:
starting_capital = 72267.75
trade_df_by_date = strategy_reporter.create_data_input_for_cumulative_returns_and_indices(final_list, index_data_dict, starting_capital)
strategy_reporter.plot_cumulative_returns_and_indices(trade_df_by_date, index_data_dict)

2025-01-22 18:34:01,163 - REPORTER - INFO - Starting trade data preparation... - performance_reporter.py:26
2025-01-22 18:34:01,220 - REPORTER - INFO - Account value validation:
                    Min: $68,998.15
                    Max: $1,125,078.05
                    Mean daily return: 0.1104%
                 - performance_reporter.py:103
2025-01-22 18:34:01,223 - REPORTER - INFO - Trade data processing completed:
                    Valid trades processed: 5197
                    Date range: 2009-01-10 23:59:59+00:00 to 2024-02-25 23:59:59+00:00
                    Starting value: $72,267.75
                    Final value: $1,119,128.26
                    Total PnL: $1,046,860.51
                 - performance_reporter.py:116
2025-01-22 18:34:01,278 - REPORTER - WARNING - Potentially unrealistic total return: 1460.41% - performance_reporter.py:384
2025-01-22 18:34:01,278 - REPORTER - WARNING - Extreme return detected: 1460.41% - performance_reporter.py:398
2025-01-22 18:34:01

In [27]:
sorted_orders[0]

[{'symbol': 'JPM',
  'timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'orderDirection': 'SELL',
  'entryPrice': 31.01,
  'orderType': 'MARKET',
  'timeInForce': 'DAY',
  'orderQuantity': 913.0,
  'strategy_name': 'pairs_trading',
  'broker': 'SIM',
  'granularity': '1d',
  'status': 'closed',
  'signal_id': '0b16492e7a28456448bada02aada656a8312ec20a5404d1d44a5084e570443ad',
  'symbol_ltp': {'2009-01-09 23:59:59+00:00': 31.01},
  'strategy_inputs': {'strategy_order_id': 'BAC_JPM_39ac903e6e684dd687bc9a376137810d'},
  'fill_price': 31.01,
  'filled_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'order_id': 'ca21370e8228469b7ea921fb3c21b07d82a48d4996c9a5b562c3de0490e71bd6',
  'broker_order_id': 'ca21370e8228469b7ea921fb3c21b07d82a48d4996c9a5b562c3de0490e71bd6',
  'message': 'Order filled at market price.',
  'fresh_update': False},
 {'symbol': 'JPM',
  'timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'orderDirection': 'BUY',
  'exitPrice': 31.6

# Plot Rolling charts

In [37]:
pair_trading_orders_dict = {}

for order_history in closed_orders:
    if order_history[0]['strategy_inputs']['strategy_order_id'] not in pair_trading_orders_dict.keys():
        pair_trading_orders_dict[order_history[0]['strategy_inputs']['strategy_order_id']] = []
    pair_trading_orders_dict[order_history[0]['strategy_inputs']['strategy_order_id']].append(
        {'symbol':order_history[0]['symbol'], 
         'direction':order_history[0]['orderDirection'],
         'entry_price':order_history[0]['fill_price'], 
         'exit_price':order_history[-1]['fill_price'], 
         'order_quantity':order_history[0]['orderQuantity'],
         'entry_timestamp':order_history[0]['filled_timestamp'], 
         'symbol_ltp-len':len(order_history[-1]['symbol_ltp']),
         'exit_timestamp':order_history[-1]['filled_timestamp'], 
         'order_value': (order_history[0]['fill_price']) * (1 if order_history[0]['orderDirection'] == 'BUY' else -1) * (order_history[0]['orderQuantity']),
         'profit': ((order_history[-1]['fill_price'] - order_history[0]['fill_price']) * (1 if order_history[0]['orderDirection'] == 'BUY' else -1) * (order_history[0]['orderQuantity']))
         })

In [38]:
pair_trading_orders_dict_ = {}
count = 0
for key, value in pair_trading_orders_dict.items():
    if len(value) == 2:
        # print(len(value))
        pair_trading_orders_dict_[key] = value
pair_trading_orders_dict = pair_trading_orders_dict_

In [50]:
both_profit_count = 0
both_loss_count = 0
mixed_count = 0
profit_count = 0
loss_count = 0
profit_pct_list = []
loss_pct_list = []

for key, value in pair_trading_orders_dict.items():
    order_1, order_2 = value
    order_1_value = abs(order_1['order_value'])
    order_2_value = abs(order_2['order_value'])
    order_1_hold_dur = order_1['symbol_ltp-len']
    order_2_hold_dur = order_2['symbol_ltp-len']
    
    order_1_profit = order_1['profit']
    order_2_profit = order_2['profit']
    total_profit = order_1_profit + order_2_profit
    if order_1_profit > 0 and order_2_profit > 0:
        both_profit_count += 1
    elif order_1_profit < 0 and order_2_profit < 0:
        both_loss_count += 1
    else:
        mixed_count += 1
    
    if total_profit > 0:
        profit_count += 1
    else:
        loss_count += 1
    pnl = total_profit/(order_1_value + order_2_value) * 100
    
    if pnl > 0:
        profit_pct_list.append(pnl)
    else:
        loss_pct_list.append(pnl)
    print(f"Metrics: order_1: {round(order_1_profit, 1)}, Order 2 PnL: {round(order_2_profit, 1)}, Total PnL: {round(total_profit, 1)}, Total Order Value: {round(order_1_value + order_2_value, 2)}, PnL %: {round(pnl, 1)}%, Order Hold Duration: {(order_1_hold_dur, order_2_hold_dur)}")

Metrics: order_1: -547.8, Order 2 PnL: 752.0, Total PnL: 204.2, Total Order Value: 45828.65, PnL %: 0.4%, Order Hold Duration: (2, 3)
Metrics: order_1: -2996.4, Order 2 PnL: 265.4, Total PnL: -2731.0, Total Order Value: 158486.06, PnL %: -1.7%, Order Hold Duration: (3, 4)
Metrics: order_1: 354.0, Order 2 PnL: 466.3, Total PnL: 820.3, Total Order Value: 43547.08, PnL %: 1.9%, Order Hold Duration: (3, 4)
Metrics: order_1: -742.7, Order 2 PnL: 308.6, Total PnL: -434.1, Total Order Value: 44033.37, PnL %: -1.0%, Order Hold Duration: (2, 3)
Metrics: order_1: -32.7, Order 2 PnL: 1648.2, Total PnL: 1615.5, Total Order Value: 41601.24, PnL %: 3.9%, Order Hold Duration: (2, 3)
Metrics: order_1: -321.4, Order 2 PnL: 1281.5, Total PnL: 960.1, Total Order Value: 51428.66, PnL %: 1.9%, Order Hold Duration: (2, 5)
Metrics: order_1: -391.3, Order 2 PnL: -55.6, Total PnL: -447.0, Total Order Value: 34927.83, PnL %: -1.3%, Order Hold Duration: (2, 2)
Metrics: order_1: -204.8, Order 2 PnL: 41.3, Total P

In [56]:
from numpy import average
average(profit_pct_list), average(loss_pct_list), len(profit_pct_list), len(loss_pct_list)

(1.763267528282061, -0.8460686312546077, 1106, 1171)

In [55]:
average(profit_pct_list)/average(loss_pct_list)

-2.084071508084833

In [46]:
both_profit_count, both_loss_count, mixed_count, profit_count, loss_count

(76, 583, 1618, 1106, 1171)

In [36]:
profit_count / (profit_count + loss_count)

0.4857268335529205

In [13]:
pair_trading_orders_dict

{'BAC_JPM_66a7c9990d58412cb5c9249743108f16': [{'symbol': 'BAC',
   'direction': 'BUY',
   'entry_price': 13.24,
   'exit_price': 14.053549482287726,
   'order_quantity': 1323.0,
   'entry_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
   'symbol_ltp-len': 6,
   'exit_timestamp': Timestamp('2009-01-15 23:59:59+0000', tz='UTC'),
   'profit': 1076.3259650666619},
  {'symbol': 'JPM',
   'direction': 'SELL',
   'entry_price': 31.01,
   'exit_price': 29.814283157228477,
   'order_quantity': 913.0,
   'entry_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
   'symbol_ltp-len': 7,
   'exit_timestamp': Timestamp('2009-01-16 23:59:59+0000', tz='UTC'),
   'profit': 1091.689477450402}],
 'XOM_CVX_04b152c5a54f43a990169293ad307aff': [{'symbol': 'CVX',
   'direction': 'BUY',
   'entry_price': 73.97,
   'exit_price': 75.14601005992478,
   'order_quantity': 718.0,
   'entry_timestamp': Timestamp('2009-01-10 23:59:59+0000', tz='UTC'),
   'symbol_ltp-len': 7,
   'exit_timestamp'

In [16]:
closed_orders[0]

[{'symbol': 'BAC',
  'timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'orderDirection': 'BUY',
  'entryPrice': 13.24,
  'orderType': 'MARKET',
  'timeInForce': 'DAY',
  'orderQuantity': 1323.0,
  'strategy_name': 'pairs_trading',
  'broker': 'SIM',
  'granularity': '1d',
  'status': 'closed',
  'signal_id': '1fc462021beda3be3201895bc934b9c5a12901717cb9ab4aba4020fc7ebba99c',
  'symbol_ltp': {'2009-01-09 23:59:59+00:00': 13.24},
  'strategy_inputs': {'strategy_order_id': 'BAC_JPM_66a7c9990d58412cb5c9249743108f16'},
  'fill_price': 13.24,
  'filled_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'order_id': 'eeb1afe4b61642d8516b4752fab9fd589e3517a7a3d320b61c454ffda93df7c8',
  'broker_order_id': 'eeb1afe4b61642d8516b4752fab9fd589e3517a7a3d320b61c454ffda93df7c8',
  'message': 'Order filled at market price.',
  'fresh_update': False},
 {'symbol': 'BAC',
  'timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'orderDirection': 'SELL',
  'exitPrice': 14.